In [15]:
import numpy as np
import cvxpy as cp

In [57]:
def estimate_params2(V, U):
    if len(V) != len(U):
        raise ValueError("Arrays must have the same size")
    T = len(V)
    #gammas = np.ones((2, 2))
    gammas = cp.Variable(2)
    constraints = [gammas >= 0.01]
    expression = 0
    for d in range(T-1):
        #expression += cp.sum_squares(np.array([V[d+1], U[d+1]]) - gammas @ np.array([V[d], U[d]]))
        expression += cp.square(V[d+1] - (U[d] + V[d]) * gammas[0])
        expression += cp.square(U[d+1] - (U[d] + V[d]) * gammas[1])
    problem = cp.Problem(cp.Minimize(expression), constraints)
    problem.solve(solver = cp.ECOS)
    
    opt_val = gammas.value
    
    return opt_val

In [ ]:
def estimate_params4(V, U):
    if len(V) != len(U):
        raise ValueError("Arrays must have the same size")
    T = len(V)
    #gammas = np.ones((2, 2))
    gammas = cp.Variable((2, 2))
    constraints = [gammas >= 0.01]
    expression = 0
    for d in range(T-1):
        expression += cp.sum_squares(np.array([V[d+1], U[d+1]]) - gammas @ np.array([V[d], U[d]]))
    problem = cp.Problem(cp.Minimize(expression), constraints)
    problem.solve(solver = cp.ECOS)
    
    opt_val = gammas.value
    
    return opt_val

In [85]:
def l1_estimate_params4(V, U, lbd):
    if len(V) != len(U):
        raise ValueError("Arrays must have the same size")
    T = len(V)
    
    gammas = cp.Variable((T, 4))
    constraints = [gammas >= 0.01]
    expression = 0
    for d in range(T-1):
        expression += cp.square(V[d+1] - gammas[d, 0] * V[d] - gammas[d, 1] * U[d])
        expression += cp.square(U[d+1] - gammas[d, 2] * V[d] - gammas[d, 3] * U[d])
        expression += lbd * cp.norm(gammas[d] - gammas[d+1], 1)
    
    problem = cp.Problem(cp.Minimize(expression), constraints)
    problem.solve(solver = cp.ECOS_BB, verbose = True)
    
    opt_val = gammas.value
    
    return opt_val

In [86]:
def l1_estimate_params2(V, U, lbd):
    if len(V) != len(U):
        raise ValueError("Arrays must have the same size")
    T = len(V)
    
    gammas = cp.Variable((T, 2))
    constraints = [gammas >= 0.01]
    expression = 0
    for d in range(T-1):
        expression += cp.square(V[d+1] - gammas[d, 0] * (U[d] + V[d]))
        expression += cp.square(U[d+1] - gammas[d, 1] * (U[d] + V[d]))
        expression += lbd * cp.norm(gammas[d] - gammas[d+1], 1)
    
    problem = cp.Problem(cp.Minimize(expression), constraints)
    problem.solve(solver = cp.ECOS_BB)
    
    opt_val = gammas.value
    
    return opt_val

In [ ]:
def delta_estimate_params4(delta, V, U):
    if len(V) != len(U):
        raise ValueError("Arrays must have the same size")
    T = len(V)
    gammas = cp.Variable((T, 4))
    constraints = [gammas >= 0.001]
    for d in range(T-1):
        constraints += [cp.norm(gammas[d+1,:]-gammas[d,:],'inf') <= delta]
    expression = 0
    for d in range(T-1):
        expression += cp.square(V[d+1] - gammas[d, 0] * V[d] - gammas[d, 1] * U[d])
        expression += cp.square(U[d+1] - gammas[d, 2] * V[d] - gammas[d, 3] * U[d])
        #expression += cp.sum_squares(np.array([V[d+1], U[d+1]]) - gammas[d,:] @ np.array([V[d], U[d]]))
    problem = cp.Problem(cp.Minimize(expression), constraints)
    problem.solve(solver = cp.ECOS, max_iters = 10000)

    opt_val = gammas.value
  
    return opt_val

In [ ]:
def delta_estimate_params2(delta, V, U):
    if len(V) != len(U):
        raise ValueError("Arrays must have the same size")
    T = len(V)
    gammas = cp.Variable((T, 2))
    constraints = [gammas >= 0.001]
    for d in range(T-1):
        constraints += [cp.norm(gammas[d+1,:]-gammas[d,:],'inf') <= delta]
    expression = 0
    for d in range(T-1):
        expression += cp.square(V[d+1] - gammas[d, 0] * (V[d] + U[d]))
        expression += cp.square(U[d+1] - gammas[d, 1] * (V[d] + U[d]))
        #expression += cp.sum_squares(np.array([V[d+1], U[d+1]]) - gammas[d,:] @ np.array([V[d], U[d]]))
    problem = cp.Problem(cp.Minimize(expression), constraints)
    problem.solve(solver = cp.ECOS, max_iters = 10000)

    opt_val = gammas.value
  
    return opt_val

In [79]:
true_gammas = np.array([0.15, 1.05])
true_gammas

array([0.15, 1.05])

In [83]:
T = 25
U = np.zeros(T)
V = np.zeros(T)
V[0] = 2
U[0] = 5
'''
for i in range(T-1):
    sigma = 0.001
    I = U[i] + V[i]
    noise1 = np.random.normal(0, sigma * np.sqrt(V[i]))
    noise2 = np.random.normal(0, sigma * np.sqrt(U[i]))
    V[i+1] = V[i] * true_gammas[0, 0] + U[i] * true_gammas[0, 1] + noise1
    U[i+1] = V[i] * true_gammas[1, 0] + U[i] * true_gammas[1, 1] + noise2
'''
for i in range(T-1):
    sigma = 1
    I = U[i] + V[i]
    noise1 = max(0, np.random.normal(0, sigma * np.sqrt(I)))
    noise2 = max(0, np.random.normal(0, sigma * np.sqrt(I)))
    V[i+1] = I * true_gammas[0] + noise1
    U[i+1] = I * true_gammas[1] + noise2


In [90]:
lbd = 100
gammaest = l1_estimate_params2(V, U, lbd)
gammaest

array([[0.21199293, 1.07465505],
       [0.21199293, 1.07465505],
       [0.21199293, 1.07465505],
       [0.21199293, 1.07465505],
       [0.21199293, 1.07465505],
       [0.23436272, 1.07465505],
       [0.24419071, 1.07465505],
       [0.17759417, 1.11722458],
       [0.16570143, 1.24391683],
       [0.20043764, 1.05790198],
       [0.15499048, 1.1688114 ],
       [0.15503571, 1.20882486],
       [0.15019412, 1.05152964],
       [0.15019412, 1.07653736],
       [0.15019412, 1.0792476 ],
       [0.15019412, 1.1027316 ],
       [0.1995738 , 1.05029754],
       [0.15019046, 1.09083338],
       [0.17269328, 1.05607521],
       [0.15008189, 1.05008189],
       [0.17476273, 1.08629526],
       [0.16141843, 1.05003575],
       [0.16857876, 1.06357255],
       [0.15000802, 1.07485545],
       [0.15000802, 1.07485545]])

Dynamic gammas generation

In [92]:
dynamic_gammas = np.empty((T, 2))

dynamic_gammas[0, 0] = 0.15
dynamic_gammas[0, 1] = 1.05

for t in range(T - 1):
    dynamic_gammas[t+1, 0] =dynamic_gammas[t, 0] + 0.005
    dynamic_gammas[t+1, 1] =dynamic_gammas[t, 1] + 0.002

In [93]:
U = np.zeros(T)
V = np.zeros(T)
V[0] = 2
U[0] = 5

for i in range(T-1):
    sigma = 0.1
    I = U[i] + V[i]
    noise1 = max(0, np.random.normal(0, sigma * np.sqrt(I)))
    noise2 = max(0, np.random.normal(0, sigma * np.sqrt(I)))
    V[i+1] = I * dynamic_gammas[i, 0] + noise1
    U[i+1] = I * dynamic_gammas[i, 1] + noise2

In [98]:
lbd = 100
gammaest = l1_estimate_params2(V, U, lbd)
gammaest

array([[0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19800241, 1.07446938],
       [0.19999999, 1.07446938],
       [0.205     , 1.07717808],
       [0.21      , 1.07717808],
       [0.2176883 , 1.07717808],
       [0.22007794, 1.07799999],
       [0.225     , 1.08383233],
       [0.23      , 1.08383233],
       [0.23914577, 1.08671663],
       [0.24      , 1.08637153],
       [0.24929108, 1.09332503],
       [0.25      , 1.09469438],
       [0.25847878, 1.09213235],
       [0.26      , 1.094     ],
       [0.26499031, 1.09649985],
       [0.26499031, 1.09649985]])

In [95]:
dynamic_gammas

array([[0.15 , 1.05 ],
       [0.155, 1.052],
       [0.16 , 1.054],
       [0.165, 1.056],
       [0.17 , 1.058],
       [0.175, 1.06 ],
       [0.18 , 1.062],
       [0.185, 1.064],
       [0.19 , 1.066],
       [0.195, 1.068],
       [0.2  , 1.07 ],
       [0.205, 1.072],
       [0.21 , 1.074],
       [0.215, 1.076],
       [0.22 , 1.078],
       [0.225, 1.08 ],
       [0.23 , 1.082],
       [0.235, 1.084],
       [0.24 , 1.086],
       [0.245, 1.088],
       [0.25 , 1.09 ],
       [0.255, 1.092],
       [0.26 , 1.094],
       [0.265, 1.096],
       [0.27 , 1.098]])

In [99]:
dynamic_gammas = np.empty((T, 2))

dynamic_gammas[0, 0] = 0.15
dynamic_gammas[0, 1] = 1.05

for t in range(T - 1):
    if t % 10 == 0:
        dynamic_gammas[t+1, 0] =dynamic_gammas[t, 0] + 0.15
        dynamic_gammas[t+1, 1] =dynamic_gammas[t, 1] + 0.10
    else:
        dynamic_gammas[t+1, 0] =dynamic_gammas[t, 0] 
        dynamic_gammas[t+1, 1] =dynamic_gammas[t, 1] 
dynamic_gammas

array([[0.15, 1.05],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.3 , 1.15],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.45, 1.25],
       [0.6 , 1.35],
       [0.6 , 1.35],
       [0.6 , 1.35],
       [0.6 , 1.35]])

In [100]:
U = np.zeros(T)
V = np.zeros(T)
V[0] = 2
U[0] = 5

for i in range(T-1):
    sigma = 0.1
    I = U[i] + V[i]
    noise1 = max(0, np.random.normal(0, sigma * np.sqrt(I)))
    noise2 = max(0, np.random.normal(0, sigma * np.sqrt(I)))
    V[i+1] = I * dynamic_gammas[i, 0] + noise1
    U[i+1] = I * dynamic_gammas[i, 1] + noise2

lbd = 100
gammaest = l1_estimate_params2(V, U, lbd)
gammaest

array([[0.30955724, 1.15773154],
       [0.30955724, 1.15773154],
       [0.30955724, 1.15773154],
       [0.30955724, 1.15773154],
       [0.30955724, 1.15773154],
       [0.30955724, 1.15773154],
       [0.30955724, 1.15773154],
       [0.30955724, 1.15094742],
       [0.30955724, 1.15094742],
       [0.30098132, 1.15094742],
       [0.30098132, 1.15094742],
       [0.45171703, 1.25      ],
       [0.45000649, 1.25264383],
       [0.45000649, 1.25008192],
       [0.4500065 , 1.25284816],
       [0.45000649, 1.25004643],
       [0.45003434, 1.25000048],
       [0.45127936, 1.25000034],
       [0.45092367, 1.25000023],
       [0.45002719, 1.25004658],
       [0.45000005, 1.25000005],
       [0.60009693, 1.3502385 ],
       [0.60001063, 1.35      ],
       [0.60002381, 1.35      ],
       [0.60002381, 1.35      ]])